## Feature Engineering + Enrichment

In [1]:
from selenium import webdriver
import concurrent.futures
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
from random import uniform
from time import sleep
import lxml
from bs4.formatter import HTMLFormatter
import re
import pandas as pd
import pickle
import os
import smtplib
from email.message import EmailMessage
import threading
from multiprocessing import Pool
import more_itertools
from selenium.webdriver.chrome.options import Options
from datetime import datetime



In [3]:
def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    # Establish a connection to Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('halpitsstockbot@gmail.com', 'ndzjdetnljjhewya')
    server.send_message(msg)
    server.quit()


In [4]:
# access most recently saved directory from the previous step
file_path = "/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Temp_Files/step_1_to_2.txt"

# Read the file and store its contents in a variable
with open(file_path, "r") as file:
    directory_path = file.read()

In [5]:
directory_path = directory_path.rstrip('/')

In [6]:
specific_run_path = directory_path.split('/')[-1]

In [7]:
specific_run_path

'2024-06-25-->2024-06-25'

In [8]:
import os
import pickle

# path to the root directory where all the pickle files are stored
all_data = []

# walk through directory_path, including subdirectories
for dirpath, dirnames, filenames in os.walk(directory_path):
    print(f"Processing directory: {dirpath}")
    for filename in filenames:
        if filename.endswith('.pkl'):
            full_filepath = os.path.join(dirpath, filename)
            print(f"  Loading file: {filename}")
            with open(full_filepath, 'rb') as file:
                data = pickle.load(file)
                all_data.append(data)  # assumes data in each pickle file is a list. Adjust as needed.

# now, all_data contains the combined data from all pickle files.


Processing directory: /Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Daily_Scrape_Pickles/2024-06-25-->2024-06-25
  Loading file: batch8.pkl
  Loading file: batch1.pkl
  Loading file: batch0.pkl
  Loading file: batch2.pkl
  Loading file: batch3.pkl
  Loading file: batch7.pkl
  Loading file: batch6.pkl
  Loading file: batch4.pkl
  Loading file: batch5.pkl


In [10]:
import pandas as pd

flattened_data = [df for sublist in all_data for df in sublist]

# iterate through each saved dataframe, format and extract what we need
for index, df in enumerate(flattened_data):
    try:
        df['code'] = df['code'].astype(str).str[0]
        df['direct_or_indirect'] = df['direct_or_indirect'].str[0]
        df['amount'] = df['amount'].astype(str).str.split('(').str[0]

        # regex pattern to match numeric values and exclude any text within parentheses
        numeric_regex = r'\$?([0-9,\.]+)'

        if len(df) > 1:  #  if more than one row
            df['total_owned_after_trans'] = df['total_owned_after_trans'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '')
            df['Price'] = df['Price'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '').str.replace('$', '')
            df['amount'] = df['amount'].astype(str).str.extract(numeric_regex).squeeze().str.replace(',', '')
            # remove content within parentheses if it exists
            df['Price'] = df['Price'].str.split('(').str[0]
        else:  # if only one row
            df['total_owned_after_trans'] = df['total_owned_after_trans'].astype(str).str.extract(numeric_regex)[0].replace(',', '')
            df['Price'] = df['Price'].astype(str).str.extract(numeric_regex)[0].replace(',', '').replace('$', '')
            df['amount'] = df['amount'].astype(str).str.extract(numeric_regex)[0].replace(',', '')
            # remove content within parentheses if it exists
            df['Price'] = df['Price'].str.split('(').str[0]
    except KeyError as e:
        print(f"KeyError encountered at DataFrame index {index}: {e}")


In [11]:
for df in flattened_data: 
    if df.shape[1] > 11:
        column_mapping = {
        df.columns[0]: 'security_type',
        df.columns[1]: 'transaction_date',
        df.columns[2]: 'execution_date',
        df.columns[3]: 'code',
        df.columns[4]: 'V',
        df.columns[5]: 'amount',
        df.columns[6]: 'a_or_d',
        df.columns[7]: 'price',
        df.columns[8]: 'amount_owned_following',
        df.columns[9]: 'd_or_i',
        df.columns[10]: 'nature_of_ownership'
        }
        df.rename(columns=column_mapping, inplace=True)
        df.columns.values[[11, 12, 13, 14]] = ['url', 'reporting_date', 'ticker','name']
        df.drop(columns=['execution_date', 'V', 'nature_of_ownership'], inplace=True)


In [12]:
#only care about outright purchases of stock by insiders
df_with_p_case_insensitive = [df for df in flattened_data if df['code'].str.lower().eq('p').any()]

In [14]:
modified_dataframes = flattened_data

In [15]:
import pandas as pd

summary_list_1 = []  # list to store the summary of each df

for index, df in enumerate(modified_dataframes):  # using enumerate to get  index
    if not df.empty:
        # print the current index
        print(f"Processing DataFrame at index: {index}")
        # clean up 'amount', 'price', and 'amount_owned_following' columns
        df['amount'] = pd.to_numeric(df['amount'].astype(str).str.replace(',', '').str.extract(r'([\d.]+)', expand=False), errors='coerce')
        df['price'] = pd.to_numeric(df['price'].astype(str).str.replace(',', '').str.extract(r'([\d.]+)', expand=False), errors='coerce')
        df['amount_owned_following'] = pd.to_numeric(df['amount_owned_following'].astype(str).str.replace(',', '').str.split('(').str[0], errors='coerce')

        # initialize net amounts
        df['total_amount_bought_or_sold'] = 0
        df['total_dollar_bought_or_sold'] = 0

        # calculate net buy/sell based on the codes
        for index, row in df.iterrows():
            code = row['code']
            amount = row['amount']
            price = row['price']
            if pd.notnull(amount) and pd.notnull(price):  # check if 'amount' and 'price' are not NaN
                dollar_amount = amount * price
                if code == 'P':
                    df.at[index, 'total_amount_bought_or_sold'] = amount
                    df.at[index, 'total_dollar_bought_or_sold'] = dollar_amount
                elif code == 'S':
                    df.at[index, 'total_amount_bought_or_sold'] = -amount
                    df.at[index, 'total_dollar_bought_or_sold'] = -dollar_amount

        # create a summary for the current df
        summary = {
            'transaction_date': df['transaction_date'].iloc[0] if 'transaction_date' in df else None,
            'total_amount_bought_or_sold': df['total_amount_bought_or_sold'].sum(),
            'total_dollar_bought_or_sold': df['total_dollar_bought_or_sold'].sum(),
            'purchase_price': df['price'].iloc[0] if 'price' in df else None,
            'amount_owned_following': df['amount_owned_following'].max() if 'amount_owned_following' in df else None,
            'url': df['url'].iloc[-1] if 'url' in df else None,
            'reporting_date': df['reporting_date'].iloc[0] if 'reporting_date' in df else None,
            'ticker': df['ticker'].iloc[0] if 'ticker' in df else None,
            'name': df['name'].iloc[0] if 'name' in df else None
        }
        summary_list_1.append(summary)

# convert the list of summaries to a df
final_summary_df_1 = pd.DataFrame(summary_list_1)



Processing DataFrame at index: 0
Processing DataFrame at index: 1
Processing DataFrame at index: 2
Processing DataFrame at index: 3
Processing DataFrame at index: 4
Processing DataFrame at index: 5
Processing DataFrame at index: 6
Processing DataFrame at index: 7
Processing DataFrame at index: 8
Processing DataFrame at index: 9
Processing DataFrame at index: 10
Processing DataFrame at index: 11
Processing DataFrame at index: 12
Processing DataFrame at index: 13
Processing DataFrame at index: 14
Processing DataFrame at index: 15
Processing DataFrame at index: 16
Processing DataFrame at index: 17
Processing DataFrame at index: 18
Processing DataFrame at index: 19
Processing DataFrame at index: 20
Processing DataFrame at index: 21
Processing DataFrame at index: 22
Processing DataFrame at index: 23
Processing DataFrame at index: 24
Processing DataFrame at index: 25
Processing DataFrame at index: 26
Processing DataFrame at index: 27
Processing DataFrame at index: 28
Processing DataFrame at 

,transaction_date,total_amount_bought_or_sold,total_dollar_bought_or_sold,purchase_price,amount_owned_following,url,reporting_date,ticker,name
0,06/24/2024,-8600.0000,-5.435200e+04,6.3200,2.783100e+04,https://sec.gov/Archives/edgar/data/882796/000...,2024-06-25,BCRX,McKee Amy E
1,06/21/2024,11675.0000,1.741354e+04,1.4500,1.874960e+05,https://sec.gov/Archives/edgar/data/883107/000...,2024-06-25,NANX,Cureton Kevin
2,06/24/2024,13812.0000,1.944730e+05,14.0800,3.429891e+06,https://sec.gov/Archives/edgar/data/884121/000...,2024-06-25,HQL,"Saba Capital Management, L.P."
3,06/21/2024,-625621.0000,-9.037770e+06,14.3600,1.811219e+07,https://sec.gov/Archives/edgar/data/894315/000...,2024-06-25,SITC,Otto Alexander
4,06/21/2024,311.0000,3.732000e+03,12.0000,1.311700e+06,https://sec.gov/Archives/edgar/data/914712/000...,2024-06-25,STCN,STEEL PARTNERS HOLDINGS L.P.
...,...,...,...,...,...,...,...,...,...
179,06/21/2024,71043.0000,6.131241e+05,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905369/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
180,06/21/2024,71043.0000,6.131241e+05,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905373/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
181,06/21/2024,71043.0000,6.131241e+05,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905374/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
182,06/24/2024(1),80.5940,8.019103e+02,9.9500,1.152389e+03,https://sec.gov/Archives/edgar/data/1910629/00...,2024-06-25,NONE,Monte Mark David


In [16]:
net_buys_1 = final_summary_df_1[final_summary_df_1['total_amount_bought_or_sold'] > 0]

In [17]:
rfb=net_buys_1

In [18]:
#concat the results of the two date fields if one is missing, prioritizing reporting_date

from dateutil.parser import parse
import pandas as pd

# function to handle strings that are not in a date format
def clean_date(date_str):
    if isinstance(date_str, str):
        if '(' in date_str:
            date_str = date_str.split('(')[0].strip()
        try:
            # try parsing the date string to see if it's in a valid date format
            parse(date_str)
            return date_str
        except ValueError:
            # If not, return NaN
            return pd.NaT
    return date_str

# apply the cleaning function to your date columns in rfb
rfb['transaction_date'] = rfb['transaction_date'].apply(clean_date)
rfb['reporting_date'] = rfb['reporting_date'].apply(clean_date)

# convert the date columns from strings to datetime objects
rfb['transaction_date'] = pd.to_datetime(rfb['transaction_date'], errors='coerce')
rfb['reporting_date'] = pd.to_datetime(rfb['reporting_date'], errors='coerce')



/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/977918013.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['transaction_date'] = rfb['transaction_date'].apply(clean_date)
/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/977918013.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['reporting_date'] = rfb['reporting_date'].apply(clean_date)
/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/977918013.py:25: SettingWithCopyWarning: 
A value is trying

In [19]:
rfb.reset_index(drop= True )

,transaction_date,total_amount_bought_or_sold,total_dollar_bought_or_sold,purchase_price,amount_owned_following,url,reporting_date,ticker,name
0,2024-06-21,11675.0000,17413.540000,1.4500,1.874960e+05,https://sec.gov/Archives/edgar/data/883107/000...,2024-06-25,NANX,Cureton Kevin
1,2024-06-24,13812.0000,194472.960000,14.0800,3.429891e+06,https://sec.gov/Archives/edgar/data/884121/000...,2024-06-25,HQL,"Saba Capital Management, L.P."
2,2024-06-21,311.0000,3732.000000,12.0000,1.311700e+06,https://sec.gov/Archives/edgar/data/914712/000...,2024-06-25,STCN,STEEL PARTNERS HOLDINGS L.P.
3,2024-06-24,184.0000,320.160000,1.7400,2.490000e+05,https://sec.gov/Archives/edgar/data/919567/000...,2024-06-25,RCG,STAHL MURRAY
4,2024-06-21,71043.0000,613124.079900,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/929351/000...,2024-06-25,LGF,Liberty 77 Capital L.P.
...,...,...,...,...,...,...,...,...,...
60,2024-06-21,71043.0000,613124.079900,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905369/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
61,2024-06-21,71043.0000,613124.079900,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905373/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
62,2024-06-21,71043.0000,613124.079900,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/1905374/00...,2024-06-25,LGF,Liberty 77 Capital L.P.
63,2024-06-24,80.5940,801.910300,9.9500,1.152389e+03,https://sec.gov/Archives/edgar/data/1910629/00...,2024-06-25,NONE,Monte Mark David


In [20]:
rfb = rfb.drop_duplicates(subset=['ticker', 'reporting_date', 'total_amount_bought_or_sold', 'name'])

In [21]:
# replace null values of transaction_date with reporting_date
rfb['transaction_date'] = rfb['transaction_date'].fillna(rfb['reporting_date'])


/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/2471129097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['transaction_date'] = rfb['transaction_date'].fillna(rfb['reporting_date'])


In [22]:
rfb.reset_index(drop=True, inplace=True)

In [23]:
rfb

,transaction_date,total_amount_bought_or_sold,total_dollar_bought_or_sold,purchase_price,amount_owned_following,url,reporting_date,ticker,name
0,2024-06-21,1.167500e+04,1.741354e+04,1.4500,1.874960e+05,https://sec.gov/Archives/edgar/data/883107/000...,2024-06-25,NANX,Cureton Kevin
1,2024-06-24,1.381200e+04,1.944730e+05,14.0800,3.429891e+06,https://sec.gov/Archives/edgar/data/884121/000...,2024-06-25,HQL,"Saba Capital Management, L.P."
2,2024-06-21,3.110000e+02,3.732000e+03,12.0000,1.311700e+06,https://sec.gov/Archives/edgar/data/914712/000...,2024-06-25,STCN,STEEL PARTNERS HOLDINGS L.P.
3,2024-06-24,1.840000e+02,3.201600e+02,1.7400,2.490000e+05,https://sec.gov/Archives/edgar/data/919567/000...,2024-06-25,RCG,STAHL MURRAY
4,2024-06-21,7.104300e+04,6.131241e+05,8.5077,9.390858e+06,https://sec.gov/Archives/edgar/data/929351/000...,2024-06-25,LGF,Liberty 77 Capital L.P.
5,2024-06-21,1.622200e+04,1.809386e+04,1.1200,1.451222e+07,https://sec.gov/Archives/edgar/data/946563/000...,2024-06-25,RVP,SHAW THOMAS J
6,2024-06-23,2.544530e+06,7.964379e+06,3.1300,5.348229e+06,https://sec.gov/Archives/edgar/data/1799260/00...,2024-06-25,NRBO,"DONG-A ST CO., LTD"
7,2024-06-20,5.000000e+05,1.492385e+06,2.9900,4.183002e+06,https://sec.gov/Archives/edgar/data/1811063/00...,2024-06-25,NUVB,Cui Xiangmin
8,2024-06-21,2.000000e+04,1.642000e+05,8.2100,9.954500e+04,https://sec.gov/Archives/edgar/data/1841968/00...,2024-06-25,PX,McCoy David M.
9,2024-06-24,4.035870e+01,4.499995e+02,11.1500,3.638320e+03,https://sec.gov/Archives/edgar/data/1845014/00...,2024-06-25,NONE,Booth Richard Whitaker


## Enrichment- add some additional company info using YahooQuery

In [24]:
import yahooquery as yq
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay as BDay
from dateutil import parser

In [25]:
rfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   transaction_date             28 non-null     datetime64[ns]
 1   total_amount_bought_or_sold  28 non-null     float64       
 2   total_dollar_bought_or_sold  28 non-null     float64       
 3   purchase_price               28 non-null     float64       
 4   amount_owned_following       28 non-null     float64       
 5   url                          28 non-null     object        
 6   reporting_date               28 non-null     datetime64[ns]
 7   ticker                       28 non-null     object        
 8   name                         28 non-null     object        
dtypes: datetime64[ns](2), float64(4), object(3)
memory usage: 2.1+ KB


In [26]:
reporting_dates = rfb['reporting_date']

In [27]:
reporting_dates

0    2024-06-25
1    2024-06-25
2    2024-06-25
3    2024-06-25
4    2024-06-25
5    2024-06-25
6    2024-06-25
7    2024-06-25
8    2024-06-25
9    2024-06-25
10   2024-06-25
11   2024-06-25
12   2024-06-25
13   2024-06-25
14   2024-06-25
15   2024-06-25
16   2024-06-25
17   2024-06-25
18   2024-06-25
19   2024-06-25
20   2024-06-25
21   2024-06-25
22   2024-06-25
23   2024-06-25
24   2024-06-25
25   2024-06-25
26   2024-06-25
27   2024-06-25
Name: reporting_date, dtype: datetime64[ns]

In [28]:
forty_five = [reporting_date + BDay(22) for reporting_date in reporting_dates]

In [29]:
minus_30 =  [reporting_date - BDay(22) for reporting_date in reporting_dates]

In [30]:
minus_10 =  [reporting_date - BDay(7) for reporting_date in reporting_dates]

In [31]:
dates = list(zip(list(rfb['ticker']), reporting_dates, forty_five, minus_10, minus_30))

In [32]:
len(dates)

28

In [33]:
import concurrent.futures
import yahooquery as yq
import numpy as np

def fetch_data_for_date(lol):
    # initialize result dictionary with all required fields
    results = {
        'vix_on_dof': None,
        'tnx_on_dof': None,
        'dof_price': None,
        'lookahead_price': None,
        'forty_five_avg': None,
        'minus_10_price': None,  # Field for minus_10 date
        'minus_30_price': None   # Field for minus_30 date
    }
    try:
        # fetch VIX and TNX data on the date of interest
        try:
            results['vix_on_dof'] = yq.Ticker('^VIX').history(start=lol[1]-BDay(2), end=lol[1])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['vix_on_dof'] = np.nan

        try:
            results['tnx_on_dof'] = yq.Ticker('^TNX').history(start=lol[1]-BDay(2), end=lol[1])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['tnx_on_dof'] = np.nan

        # fetch stock prices for the main date and lookahead date
        holder = []
        try:
            holder = yq.Ticker(lol[0]).history(start=lol[1], end=lol[2])['close']
        except (IndexError, KeyError, AttributeError):
            pass

        try:
            results['dof_price'] = np.mean(holder[:3])
        except (IndexError, KeyError, AttributeError):
            results['dof_price'] = np.nan

        try:
            results['lookahead_price'] = np.mean(holder[-3:])
        except (IndexError, KeyError, AttributeError):
            results['lookahead_price'] = np.nan

        # calculate 45-day average
        try:
            results['forty_five_avg'] = holder.mean()
        except (IndexError, KeyError, AttributeError):
            results['forty_five_avg'] = np.nan

        # fetch stock price for minus_10
        try:
            results['minus_10_price'] = yq.Ticker(lol[0]).history(start=lol[3]-BDay(2), end=lol[3])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['minus_10_price'] = np.nan

        # fetch stock price for minus_30
        try:
            results['minus_30_price'] = yq.Ticker(lol[0]).history(start=lol[4]-BDay(2), end=lol[4])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['minus_30_price'] = np.nan
    
    except (IndexError, KeyError, AttributeError, TypeError) as e:
        results['vix_on_dof'] = np.nan
        results['tnx_on_dof'] = np.nan
        results['dof_price'] = np.nan
        results['lookahead_price'] = np.nan
        results['forty_five_avg'] = np.nan
        results['minus_10_price'] = np.nan
        results['minus_30_price'] = np.nan
            
    return results

# lists to store results
dof_price = []
lookahead_price = []
forty_five_avg = []
vix_on_dof = []
tnx_on_dof = []
minus_10_price = []
minus_30_price = [] 


with concurrent.futures.ThreadPoolExecutor() as executor:
    for index, result in enumerate(executor.map(fetch_data_for_date, dates)):
        dof_price.append(result['dof_price'])
        lookahead_price.append(result['lookahead_price'])
        forty_five_avg.append(result['forty_five_avg'])
        vix_on_dof.append(result['vix_on_dof'])
        tnx_on_dof.append(result['tnx_on_dof'])
        minus_10_price.append(result['minus_10_price'])
        minus_30_price.append(result['minus_30_price'])
        print("Processed index:", index + 1)

print('all done')


Processed index: 1
Processed index: 2
Processed index: 3
Processed index: 4
Processed index: 5
Processed index: 6
Processed index: 7
Processed index: 8
Processed index: 9
Processed index: 10
Processed index: 11
Processed index: 12
Processed index: 13
Processed index: 14
Processed index: 15
Processed index: 16
Processed index: 17
Processed index: 18
Processed index: 19
Processed index: 20
Processed index: 21
Processed index: 22
Processed index: 23
Processed index: 24
Processed index: 25
Processed index: 26
Processed index: 27
Processed index: 28
all done


In [34]:
rfb['dof_price'] = dof_price
rfb['30_day_price'] = lookahead_price
rfb['avg_30_price'] = forty_five_avg
rfb['vix_on_dof'] = vix_on_dof
rfb['tnx_on_dof'] = tnx_on_dof
rfb['minus_10_price'] = minus_10_price
rfb['minus_30_price'] = minus_30_price

/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/211787828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['dof_price'] = dof_price
/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/211787828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['30_day_price'] = lookahead_price
/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/211787828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [35]:
# accounting for very small edge case where the company hasn't been around for 30 days yet. 
import numpy as np

# for rows where minus_30_price is NaN and minus_10_price is not NaN, replace minus_30_price with minus_10_price
rfb['minus_30_price'] = np.where(rfb['minus_30_price'].isna() & rfb['minus_10_price'].notna(), 
                                 rfb['minus_10_price'], 
                                 rfb['minus_30_price'])

# for rows where both minus_30_price and minus_10_price are NaN but dof_price isn't, 
# replace both minus_30_price and minus_10_price with dof_price
condition = rfb['minus_30_price'].isna() & rfb['minus_10_price'].isna() & rfb['dof_price'].notna()
rfb.loc[condition, 'minus_30_price'] = rfb['dof_price']
rfb.loc[condition, 'minus_10_price'] = rfb['dof_price']


/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/54852664.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['minus_30_price'] = np.where(rfb['minus_30_price'].isna() & rfb['minus_10_price'].notna(),


In [36]:
rfb['ticker'] = rfb['ticker'].str.strip()

/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/1016146079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['ticker'] = rfb['ticker'].str.strip()


In [37]:
rfb['shares_percent_increased'] = rfb['total_amount_bought_or_sold']/ (abs(rfb['total_amount_bought_or_sold']) + rfb['amount_owned_following'])

/var/folders/k_/c3q4snr522d_bn076xmd5q100000gn/T/ipykernel_26430/1761103943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['shares_percent_increased'] = rfb['total_amount_bought_or_sold']/ (abs(rfb['total_amount_bought_or_sold']) + rfb['amount_owned_following'])


In [38]:
rfb = rfb.drop_duplicates()

In [39]:
rfb.dropna(inplace=True)

In [40]:
rfb.reset_index(inplace = True, drop= True)

In [41]:
rfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   transaction_date             23 non-null     datetime64[ns]
 1   total_amount_bought_or_sold  23 non-null     float64       
 2   total_dollar_bought_or_sold  23 non-null     float64       
 3   purchase_price               23 non-null     float64       
 4   amount_owned_following       23 non-null     float64       
 5   url                          23 non-null     object        
 6   reporting_date               23 non-null     datetime64[ns]
 7   ticker                       23 non-null     object        
 8   name                         23 non-null     object        
 9   dof_price                    23 non-null     float64       
 10  30_day_price                 23 non-null     float64       
 11  avg_30_price                 23 non-null     fl

In [42]:
rfb['10_day_delta'] = (rfb['dof_price']- rfb['minus_10_price']) / rfb['minus_10_price']
rfb['30_day_delta'] = (rfb['dof_price']- rfb['minus_30_price']) / rfb['minus_30_price']

In [43]:
rfb['dof_vs_30_avg'] = (rfb['avg_30_price'] - rfb['dof_price']) / rfb['dof_price']
rfb['dof_vs_30_actual'] = (rfb['30_day_price'] - rfb['dof_price']) / rfb['dof_price']

In [44]:
rfb['purchase_price_vs_price_on_dof'] = ((rfb['purchase_price'] - rfb['dof_price']) / rfb['dof_price'])*100

In [45]:
engineered_df = rfb[['ticker',
                     'reporting_date',
                     'url',
                     'total_amount_bought_or_sold',
                     'total_dollar_bought_or_sold',
                     'amount_owned_following',
                     'purchase_price',
                     'shares_percent_increased',
                     'dof_price',
                     'vix_on_dof',
                     'tnx_on_dof',
                     'minus_10_price',
                     '10_day_delta',
                     'purchase_price_vs_price_on_dof',
                     '30_day_delta',
                     'minus_30_price',
                      'dof_vs_30_avg',
                     'dof_vs_30_actual'
                     
                     
                     
                     
                     
    
]]

In [46]:
engineered_df

,ticker,reporting_date,url,total_amount_bought_or_sold,total_dollar_bought_or_sold,amount_owned_following,purchase_price,shares_percent_increased,dof_price,vix_on_dof,tnx_on_dof,minus_10_price,10_day_delta,purchase_price_vs_price_on_dof,30_day_delta,minus_30_price,dof_vs_30_avg,dof_vs_30_actual
0,NANX,2024-06-25,https://sec.gov/Archives/edgar/data/883107/000...,11675.0,17413.54,1.874960e+05,1.4500,0.058618,1.550000,13.265,4.2525,1.485000,0.043771,-6.451610,0.127273,1.375000,0.0,0.0
1,HQL,2024-06-25,https://sec.gov/Archives/edgar/data/884121/000...,13812.0,194472.96,3.429891e+06,14.0800,0.004011,14.190000,13.265,4.2525,13.880000,0.022334,-0.775191,0.048393,13.535000,0.0,0.0
2,STCN,2024-06-25,https://sec.gov/Archives/edgar/data/914712/000...,311.0,3732.00,1.311700e+06,12.0000,0.000237,12.800000,13.265,4.2525,12.290000,0.041497,-6.250001,0.110629,11.525000,0.0,0.0
3,RCG,2024-06-25,https://sec.gov/Archives/edgar/data/919567/000...,184.0,320.16,2.490000e+05,1.7400,0.000738,1.660000,13.265,4.2525,1.715000,-0.032070,4.819279,0.027864,1.615000,0.0,0.0
4,RVP,2024-06-25,https://sec.gov/Archives/edgar/data/946563/000...,16222.0,18093.86,1.451222e+07,1.1200,0.001117,1.090000,13.265,4.2525,1.150000,-0.052174,2.752290,0.178378,0.925000,0.0,0.0
5,NRBO,2024-06-25,https://sec.gov/Archives/edgar/data/1799260/00...,2544530.0,7964378.90,5.348229e+06,3.1300,0.322388,4.260000,13.265,4.2525,3.880500,0.097797,-26.525826,0.024654,4.157500,0.0,0.0
6,NUVB,2024-06-25,https://sec.gov/Archives/edgar/data/1811063/00...,500000.0,1492385.37,4.183002e+06,2.9900,0.106769,3.040000,13.265,4.2525,2.955000,0.028765,-1.644736,-0.044025,3.180000,0.0,0.0
7,PX,2024-06-25,https://sec.gov/Archives/edgar/data/1841968/00...,20000.0,164200.00,9.954500e+04,8.2100,0.167301,8.250000,13.265,4.2525,8.065000,0.022939,-0.484848,0.009174,8.175000,0.0,0.0
8,BATRA,2024-06-25,https://sec.gov/Archives/edgar/data/1185533/00...,2000.0,79920.00,4.200000e+04,39.9600,0.045455,40.599998,13.265,4.2525,41.955000,-0.032297,-1.576351,-0.004780,40.795000,0.0,0.0
9,NXDT,2024-06-25,https://sec.gov/Archives/edgar/data/1228922/00...,40471.0,197903.19,4.281441e+06,4.8900,0.009364,5.100000,13.265,4.2525,4.920000,0.036585,-4.117645,-0.045837,5.345000,0.0,0.0


In [47]:
final = engineered_df

In [48]:
final = final.dropna()

In [49]:
final.reset_index(inplace=True, drop=True)

In [50]:
folder_path = f"/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Cleaned_Enriched/{specific_run_path}"

# check if the folder exists, and create it if it doesn't
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created: {folder_path}")
else:
    print(f"Folder already exists: {folder_path}")

Folder created: /Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Cleaned_Enriched/2024-06-25-->2024-06-25


In [51]:
csv_path = f"{folder_path}/{specific_run_path}.csv"

final.to_csv(path_or_buf=csv_path)

In [52]:
# create temp file to reference in next step
file_path = "/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Temp_Files/step_2_to_3.txt"


# create or overwrite the file and write the contents of the string variable to it
with open(file_path, "w") as file:
    file.write(csv_path)
